In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark aggregation functions") \
    .getOrCreate()

25/08/17 15:19:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
listings = spark.read.csv("./data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)
listings.printSchema()

[Stage 1:>                                                          (0 + 1) / 1]

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [5]:
reviews = spark.read.csv("./data/reviews.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)
reviews.printSchema()

[Stage 3:>                                                          (0 + 1) / 1]

root
 |-- listing_id: long (nullable = true)
 |-- id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [80]:
# 1. Count the number of reviews per listing using the "reviews" dataset
# listings_reviews = listings.join(reviews, listings.id == reviews.listing_id, how="inner")
# listings_reviews\
#     .groupBy(listings.id, listings.name)\
#     .count()\
#     .show(truncate=False)
#
# v technically right answer ^ answer with uneccesary join
#
reviews_per_listing = reviews\
    .groupBy('listing_id')\
    .count()\
    .show(10)

[Stage 202:>                                                        (0 + 1) / 1]

+----------+-----+
|listing_id|count|
+----------+-----+
|     78606|    2|
|    444886|   12|
|    466017|   28|
|   2736493|    4|
|   2557853|   89|
|   3132302|    3|
|   3917692|    1|
|   3734796|    5|
|   3997029|    7|
|   4361078|   70|
+----------+-----+
only showing top 10 rows


In [24]:
import pyspark.sql.functions as F

In [31]:
listings.select(listings.review_scores_rating).limit(5).show()

+--------------------+
|review_scores_rating|
+--------------------+
|                4.68|
|                4.58|
|                 4.5|
|                4.64|
|                4.68|
+--------------------+



In [83]:
# 2. Compute the total number of listings and average review score per host
# Total num of listings
avg_review_score = listings\
    .filter(listings.review_scores_rating.isNotNull())\
    .groupBy(listings.host_id)\
    .agg(
        F.count(listings.id).alias("num_of_listings"),
        F.avg(listings.review_scores_rating).alias("avg_rev_score")
    )\
    .orderBy("num_of_listings", ascending=[False])\
    .show()
avg_review_score

[Stage 211:>                                                        (0 + 1) / 1]

+---------+---------------+------------------+
|  host_id|num_of_listings|     avg_rev_score|
+---------+---------------+------------------+
| 28820321|            258| 4.580697674418607|
|  1432477|            217| 4.400184331797233|
|314162972|            201| 4.373432835820895|
| 33889201|            120| 4.634916666666666|
|124359784|            118|4.4044915254237305|
| 83740964|            115|4.4220000000000015|
|228928499|            112| 4.846607142857143|
| 47609036|            109|  4.75165137614679|
|590452007|            109| 4.114311926605505|
| 30253178|            108| 4.475185185185184|
| 89355192|            101|  4.64990099009901|
| 74167394|             98| 4.086122448979591|
|156158778|             94|  4.89031914893617|
|538103588|             88| 4.827727272727272|
|519945528|             84| 4.037261904761904|
| 92967793|             84|4.7767857142857135|
|  7701687|             76| 4.498552631578947|
|215357262|             76| 4.514342105263158|
| 11515631|  

In [85]:
# 3: Find the top ten listings with the highest number of reviews
listings\
    .select("name", "number_of_reviews")\
    .orderBy("number_of_reviews", ascending=[False])\
    .limit(10)\
    .show(truncate=False)

## v ^ Either works

# reviews\
#     .groupBy('listing_id')\
#     .count()\
#     .orderBy('count', ascending=False)\
#     .limit(10)\
#     .show()

[Stage 217:>                                                        (0 + 1) / 1]

+--------------------------------------------------+-----------------+
|name                                              |number_of_reviews|
+--------------------------------------------------+-----------------+
|Double Room+ Ensuite                              |1855             |
|Double Garden View room - London House Hotel***   |1682             |
|Private double room with en suite facilities      |1615             |
|Locke Studio Apartment at Leman Locke             |1436             |
|Budget Double Room In Colliers Hotel.             |1433             |
|Cosy Double in Kings Cross Houseshare nr Eurostar |1195             |
|London's best transport hub 5 mins walk! Safe too!|1122             |
|En-suite Double in Kings Cross Houseshare Eurostar|1005             |
|KX Basic- Small Double- shared bathroom           |978              |
|Double in Kings Cross Houseshare nr Eurostar      |973              |
+--------------------------------------------------+-----------------+



In [93]:
# 4. Find the top five neighborhoods with the most listings
listings\
    .filter(listings.neighbourhood_cleansed.isNotNull())\
    .groupBy(listings.neighbourhood_cleansed)\
    .agg(
        F.count(listings.id).alias('num_listings')
    )\
    .orderBy('num_listings', ascending=[False])\
    .limit(5)\
    .show(truncate=False)

# Both right, ^ Mine v his

# listings \
#     .groupBy('neighbourhood_cleansed')\
#     .count() \
#     .orderBy('count', ascending=False)\
#     .limit(5)\
#     .show()

[Stage 230:>                                                        (0 + 1) / 1]

+----------------------+------------+
|neighbourhood_cleansed|num_listings|
+----------------------+------------+
|Westminster           |11367       |
|Tower Hamlets         |7566        |
|Camden                |6564        |
|Kensington and Chelsea|6348        |
|Hackney               |6279        |
+----------------------+------------+



In [51]:
# 5. Get a data frame with the following four columns:
# * Listing's ID
# * Listing's name
# * Reviewer's name
# * Review's comment
# Use "join" to combine data from two datasets
l_r_2 = listings\
    .join(reviews, listings.id == reviews.listing_id, how="inner")
lr_dr = l_r_2\
    .select(listings.id, l_r_2.name, l_r_2.reviewer_name, l_r_2.comments)\
    .show(20)
lr_dr

[Stage 122:>                                                        (0 + 1) / 1]

+-----+--------------------+-------------+--------------------+
|   id|                name|reviewer_name|            comments|
+-----+--------------------+-------------+--------------------+
|13913|Holiday London DB...|      Michael|My girlfriend and...|
|13913|Holiday London DB...|      Mathias|Alina was a reall...|
|13913|Holiday London DB...|      Kristin|Alina is an amazi...|
|13913|Holiday London DB...|      Camilla|Alina's place is ...|
|13913|Holiday London DB...|        Jorik|Nice location in ...|
|13913|Holiday London DB...|         Vera|I'm very happy to...|
|13913|Holiday London DB...|         Honi|I stayed with Ali...|
|13913|Holiday London DB...|   Alessandro|Alina was a perfe...|
|13913|Holiday London DB...|         Oleh|Alina's flat is e...|
|13913|Holiday London DB...|           Mo|The House is a pi...|
|13913|Holiday London DB...|            A|Was great base fo...|
|13913|Holiday London DB...|       Daniel|Alina was an amaz...|
|13913|Holiday London DB...|      Belind

In [94]:
# 6.Get top five listings with the highest average review comment length. 
# Only return listings with at least 5 reviews
# Use the "length" function from the "pyspark.sql.functions"
# to get a lenght of a review

# l_r_2\
#     .groupBy(listings.id, listings.name, reviews.comments)\
#     .agg(
#         F.count(reviews.id).alias("num_of_reviews"),
#         F.avg(F.length(reviews.comments).alias("rev_com_length"))
#     )\
#     .filter("num_of_reviews > 50")\
#     .orderBy("num_of_reviews", ascending=[False])\
#     .show()

l_r_2\
    .groupBy(listings.id, listings.name, listings.number_of_reviews)\
    .agg(
        F.avg(F.length(l_r_2.comments)).alias("avg_rev_com_length")
    )\
    .filter(listings.number_of_reviews > 5)\
    .orderBy("avg_rev_com_length", ascending=[False])\
    .limit(5)\
    .show()

## ^mine v his - both right

# from pyspark.sql.functions import length, avg, count

# reviews_with_comment_length = reviews.withColumn('comment_length', length('comments'))
# reviews_with_comment_length \
#   .join(listings, reviews_with_comment_length.listing_id == listings.id, 'inner') \
#   .groupBy('listing_id').agg(
#       avg(reviews_with_comment_length.comment_length).alias('average_comment_length'),
#       count(reviews_with_comment_length.id).alias('reviews_count')
#   ) \
#   .filter('reviews_count >= 5') \
#   .orderBy('average_comment_length', ascending=False) \
#   .show()

[Stage 234:>                                                        (0 + 1) / 1]

+------------------+--------------------+-----------------+------------------+
|                id|                name|number_of_reviews|avg_rev_com_length|
+------------------+--------------------+-----------------+------------------+
|618608352812465378|Beautiful Georgia...|                6|1300.1666666666667|
|          28508447|The warm and cosy...|                6|1089.3333333333333|
|627425975703032358|Superb loft beaut...|                9| 951.7777777777778|
|            979753|Pop-Art Brand New...|               13| 893.9230769230769|
|630150178279666225|Georgian oasis of...|               11| 890.7272727272727|
+------------------+--------------------+-----------------+------------------+



In [95]:
# 7. Using the "join" operator find listings without reviews.
# Hint: Use "left_join" or "left_anti" join type when implementing this

# new_join = listings.join(reviews, listings.id == reviews.listing_id, how="left_anti")
# new_join.select(new_join.comments).limit(5).show()

# His solution one
# joined_df = listings.join(
#     reviews,
#     listings.id == reviews.listing_id,
#     how='left_outer'
# )

# joined_df \
#   .filter(reviews.listing_id.isNull()) \
#   .select('name') \
#   .show(truncate=False)

# His solution two
listings_without_reviews = listings \
  .join(
    reviews,
    listings.id == reviews.listing_id,
    how='left_anti'
  ) \
  .select('name') \
  .show(truncate=False)

[Stage 241:>                                                        (0 + 1) / 1]

+------------------------------------------------+
|name                                            |
+------------------------------------------------+
|ChiqDoube Room in PrivateAppartment             |
|ROOM TO RENT IN THE OLYMPIC PERIOD              |
|4 bed Beautiful west london home                |
|London, Hoxton. Nice, 2 bedroom, 7th floor flat.|
|Bright Dbl/Nr/ Excellnt Transp                  |
|Stunning Shared Penthouse Apartment             |
|The Old Coach House (Olympics)                  |
|Studio 20min Walk from Olympic City             |
|Luxury single room                              |
|Contemporary house London E4                    |
|A lovely one bedroom garden flat!!              |
|Coming to London for the Olympics?              |
|Lovely double room + own bathroom               |
|Double Room close to Olympic Park!              |
|Double bedroom near Olympic Park                |
|SPARE ROOM TO LET DURING OLYMPICS               |
|Your Studio Flat in London-Oly